In [17]:
import toml
from scipy.constants import R
from moleculegraph.funcs import read_pair_potentials, read_bond_potentials, read_angle_potentials, read_torsion_potentials

# Write force field toml files #

This notebook is used to translate the TAMie force field files into a moleculegraph compatible toml file. The units are also converted to LAMMPS standard units (kcal/mol).

In [18]:
# Define the TAMie force field folder
path_to_tamie    = "../force-fields/UA_TAMie/"

# Define the output toml file
output_file_name = "../force-fields/forcefield_full_xx.toml"

# Read in the force field files using moleculegraph

pair_pots    = read_pair_potentials( path_to_tamie + "pair_potentials" )
bond_pots    = read_bond_potentials( path_to_tamie + "bond_potentials" )  
angle_pots   = read_angle_potentials( path_to_tamie + "angle_potentials" )
torsion_pots = read_torsion_potentials( path_to_tamie + "torsion_potentials" )

# Converstion factor from unit Kelving to kcal/mol (K * J/(K*mol) * kcal/J)
KB_kcal = R / 4184

# Define pair interactions (sigma in Angstrom, epsilon in kcal/mol)
for p in pair_pots:
    pair_pots[p]["style"]   = "hybrid Mie/cut coul/long"                
    pair_pots[p]["epsilon"] = round( pair_pots[p]["epsilon"] * KB_kcal, 4 )

# Define bonded interactions (Spring constant in kcal/mol / Angstrom^2. As TAMie utilize fixed bond, use empirical value: https://doi.org/10.1021/acs.jced.9b01031 )
for p in bond_pots:
    bond_pots[p]["style"]        = "harmonic"
    bond_pots[p]["p"][1]         = 200.

# Define angle interactions (Spring constant in kcal/mol / rad^2) --> introduce the factor 1/2 in the constant (see LAMMPS documentary)
for p in angle_pots:
    angle_pots[p]["style"]       = "harmonic"
    angle_pots[p]["p"][1]        = round (angle_pots[p]["p"][1] * KB_kcal/2, 4 )

# Define dihedral interactions (Spring constant in kcal/mol) 
for p in torsion_pots:
    if torsion_pots[p]["type"] == 1:
        torsion_pots[p]["style"]     = "opls"
        torsion_pots[p]["p"]         = [ round( x * KB_kcal * 2, 4 ) for x in torsion_pots[p]["p"] ]

    elif torsion_pots[p]["type"] == 6:
        torsion_pots[p]["style"]     = "nharmonic"
        torsion_pots[p]["p"]         = [ round( x * KB_kcal, 4 ) for x in torsion_pots[p]["p"] ]
    

# Write toml file
forcefield = {}
forcefield["atoms"]    = pair_pots
forcefield["bonds"]    = bond_pots
forcefield["angles"]   = angle_pots
forcefield["torsions"] = torsion_pots


with open(output_file_name, "w") as toml_file:
    toml.dump(forcefield, toml_file)